In [ ]:
import pandas as pd
import os
import json
import re

In [ ]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

In [ ]:
### Read Train Data
df_train=pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
df_train.head()

In [ ]:
## Read Test Data
df_test=pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
df_test.head()

In [ ]:
## Read Train Jsons to extract Text 
df_input = pd.DataFrame(columns=['id','section_title','text','data_label'])
ids=df_train['Id'].values
for id in ids:
    df=pd.read_json ('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(id))
    
    for data_label in df_train[df_train['Id']==id]['dataset_label'].values:
        new_df=df[df['text'].str.contains(data_label)].copy(deep=True)
        new_df.loc[:,['data_label']] = data_label
        new_df.loc[:,['id']] = id
        new_df.reset_index(inplace=True,drop=True)
        df_input=pd.concat([df_input, new_df], ignore_index=True)
        df_input.reset_index(inplace=True,drop=True)


In [ ]:
df_input.head()

In [ ]:
## Read Test Jsons
df_test_input = pd.DataFrame(columns=['id','section_title','text'])
ids=df_test['Id'].values
for id in ids:
    df=pd.read_json ('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(id))
    
    df.loc[:,['id']] = id
    df.reset_index(inplace=True,drop=True)
    df_test_input=pd.concat([df_test_input, df], ignore_index=True)
    df_test_input.reset_index(inplace=True,drop=True)

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)
submission_df.head()

In [ ]:
## Create list of target Dataset Labels
datasets_titles = [str(x).lower() for x in df_input['data_label'].unique()]
labels = []

In [ ]:
### String matching if Dataset label is present in Journal Text
for index in submission_df.index:
    publication_text = df_test_input[df_test_input['id'] == index].text.str.cat(sep='\n').lower()
    
    label = []
    for dataset_title in datasets_titles:
        if dataset_title in publication_text:
            label.append(clean_text(dataset_title))
    labels.append('|'.join(label))

In [ ]:
### Replace Nulls with top 2 frequent dataset labels in Training set
submission_df['PredictionString'] = labels
submission_df['PredictionString'].fillna('alzheimer s disease neuroimaging initiative adni|adni', inplace=True)
submission_df.to_csv('submission.csv')

submission_df